In [1]:
import tensorflow as tf
import os
import random
import shutil

tf.random.set_seed(1)
random.seed(1)

In [2]:
PATH ='./data/dogs-vs-cats/'

train_dir = os.path.join(PATH, 'train')
valid_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

In [3]:
train_len = len(os.listdir(train_dir))
valid_len = len(os.listdir(valid_dir))
test_len = len(os.listdir(test_dir))

In [4]:
print('total training images:', train_len)
print('total validation images:', valid_len)
print('total testing images:', test_len)

total training images: 20000
total validation images: 5000
total testing images: 12500


In [5]:
# def create_validation_sets():
#     shutil.rmtree(valid_dir)
#     os.mkdir(valid_dir)
#     shutil.rmtree(train_dir)
#     os.mkdir(train_dir)
    
#     valid_images = random.sample(os.listdir(original_dir), int(train_len * 0.20))
    
#     for image in valid_images:
#         shutil.move(os.path.join(original_dir, image), valid_dir)

# create_validation_sets()

In [6]:
CLASS_NAMES = ["cat", "dog"]

EPOCH_SIZE = 5
BATCH_SIZE = 32
BUFFER_SIZE= BATCH_SIZE * 5
STEPS_PER_EPOCH = train_len//BATCH_SIZE
VALIDATION_STEPS = valid_len//BATCH_SIZE
IMG_HEIGHT = 128
IMG_WIDTH = 128

In [7]:
def process_label(path):
    label = tf.strings.split(path, os.path.sep)[-1]
    label = tf.strings.split(label, ".")[0]
    return tf.cast(label == CLASS_NAMES, tf.float32)

def process_image(path):
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])

def scale_image(image):
    image = tf.image.per_image_standardization(image)
    return image

def process_data(path):
    image = process_image(path)
    image = scale_image(image)
    label = process_label(path)
    
    return image, label

def input_fn(path):
    list_ds = tf.data.Dataset.list_files(str(path))
    return list_ds \
        .map(process_data) \
        .shuffle(BUFFER_SIZE) \
        .batch(BATCH_SIZE) \
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE) \
        .repeat(EPOCH_SIZE)

In [8]:
train_ds = input_fn(os.path.join(train_dir, "*.jpg")).take(5000)
valid_ds = input_fn(os.path.join(valid_dir, "*.jpg")).take(2000)
test_ds = input_fn(os.path.join(test_dir, "*.jpg")).take(5000)

W0424 13:59:40.514496 140736099853184 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/image_ops_impl.py:1518: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
for (trainX, trainY) in train_ds.take(5):
    print(trainX.shape)

(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)
(32, 128, 128, 3)


In [11]:
def model_fn():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        tf.keras.layers.MaxPool2D(strides=2),
        tf.keras.layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'),
        tf.keras.layers.MaxPool2D(strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(84, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer=tf.optimizers.Adam(lr=5e-4),
                  loss=tf.losses.CategoricalCrossentropy(), 
                  metrics=["accuracy"])
    
    return model

In [12]:
model = model_fn()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 48)        38448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 48)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43200)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               11059456  
_________________________________________________________________
dense_1 (Dense)              (None, 84)                2

In [ ]:
history = model.fit(train_ds,
                    validation_data=valid_ds,
                    validation_steps=int(2000 / BATCH_SIZE),
                    epochs = EPOCH_SIZE, 
                    steps_per_epoch = int(5000 / BATCH_SIZE), 
                    use_multiprocessing=True, 
                    workers=7)

Train for 156 steps, validate for 62 steps
Epoch 1/5
156/156 [==============================] - 83s 531ms/step - loss: 0.6930 - accuracy: 0.5946 - val_loss: 0.6067 - val_accuracy: 0.6522
Epoch 2/5
156/156 [==============================] - 93s 595ms/step - loss: 0.6000 - accuracy: 0.6679 - val_loss: 0.5746 - val_accuracy: 0.6910
Epoch 3/5
156/156 [==============================] - 101s 645ms/step - loss: 0.5598 - accuracy: 0.7053 - val_loss: 0.5363 - val_accuracy: 0.7434
Epoch 4/5
156/156 [==============================] - 94s 601ms/step - loss: 0.5394 - accuracy: 0.7244 - val_loss: 0.5141 - val_accuracy: 0.7460
Epoch 5/5
155/156 [============================>.] - ETA: 0s - loss: 0.4831 - accuracy: 0.7673